# Debugging autoreload

In [ ]:
%load_ext autoreload
%autoreload 2

# Load packages

In [ ]:
from pytorch_tabular.utils import load_covertype_dataset
from rich.pretty import pprint
from sklearn.model_selection import BaseCrossValidator, ParameterGrid, ParameterSampler
import torch
import pickle
import shutil
import shap
from sklearn.model_selection import RepeatedStratifiedKFold
from glob import glob
import ast
import matplotlib.pyplot as plt
import seaborn as sns
import copy
from sklearn.model_selection import train_test_split
import numpy as np
from pytorch_tabular.utils import make_mixed_dataset, print_metrics
from pytorch_tabular import available_models
from pytorch_tabular import TabularModel
from pytorch_tabular.models import CategoryEmbeddingModelConfig, GANDALFConfig, TabNetModelConfig, FTTransformerConfig, DANetConfig
from pytorch_tabular.config import DataConfig, OptimizerConfig, TrainerConfig
from pytorch_tabular.models.common.heads import LinearHeadConfig
from pytorch_tabular.tabular_model_tuner import TabularModelTuner
from torchmetrics.functional.regression import mean_absolute_error, pearson_corrcoef
from pytorch_tabular import MODEL_SWEEP_PRESETS
import pandas as pd
from pytorch_tabular import model_sweep
from src.pt.model_sweep import model_sweep_custom
import warnings
from src.utils.configs import read_parse_config
from src.utils.hash import dict_hash
from src.pt.hyper_opt import train_hyper_opt
import pathlib
from tqdm import tqdm
import distinctipy
import matplotlib.patheffects as pe
import matplotlib.colors as mcolors
from statannotations.Annotator import Annotator
from scipy.stats import mannwhitneyu
from regression_bias_corrector import LinearBiasCorrector
import optuna
from sklearn.preprocessing import LabelEncoder
from plottable import ColumnDefinition, Table
from plottable.plots import bar
from plottable.cmap import normed_cmap, centered_cmap
import matplotlib.lines as mlines
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import scipy.stats
from functools import reduce
from scipy import stats
import shutil
import os
from pypdf import PdfReader
import re
import logging
from slugify import slugify

logging.getLogger("pypdf").setLevel(logging.ERROR)


def make_rgb_transparent(rgb, bg_rgb, alpha):
    return [alpha * c1 + (1 - alpha) * c2 for (c1, c2) in zip(rgb, bg_rgb)]

# Load components

In [ ]:
dir_root = f"E:/Git/MillenniumAge"

feat_trgt = 'Возраст'

components = {
    'Оценка состава тела, женщины': {
        'name': 'Оценка состава тела',
        'path': f"{dir_root}/data/Оценка состава тела/females",
        'bkg_count': 30,
        'likelihood': 1.00
    },
    'Оценка состава тела, мужчины': {
        'name': 'Оценка состава тела',
        'path': f"{dir_root}/data/Оценка состава тела/males",
        'bkg_count': 15,
        'likelihood': 1.00
    },
    
    'Электрокардиограмма, все': {
        'name': 'Электрокардиограмма',
        'path': f"{dir_root}/data/Электрокардиограмма/all",
        'bkg_count': 17,
        'likelihood': 0.50
    },
    'Электрокардиограмма, старше 15': {
        'name': 'Электрокардиограмма',
        'path': f"{dir_root}/data/Электрокардиограмма/over15",
        'bkg_count': 15,
        'likelihood': 0.50
    },
    'Электрокардиограмма, младше 15': {
        'name': 'Электрокардиограмма',
        'path': f"{dir_root}/data/Электрокардиограмма/under15",
        'bkg_count': 6,
        'likelihood': 0.50
    },
    
    'Гематологические исследования, все': {
        'name': 'Гематологические исследования',
        'path': f"{dir_root}/data/Гематологические исследования/all",
        'bkg_count': 30,
        'likelihood': 0.60
    },
    'Гематологические исследования, старше 15': {
        'name': 'Гематологические исследования',
        'path': f"{dir_root}/data/Гематологические исследования/over15",
        'bkg_count': 30,
        'likelihood': 0.60
    },
    'Гематологические исследования, младше 15': {
        'name': 'Гематологические исследования',
        'path': f"{dir_root}/data/Гематологические исследования/under15",
        'bkg_count': 20,
        'likelihood': 0.60
    },
    
    'Биохимические исследования 7, все': {
        'name': 'Биохимические исследования',
        'path': f"{dir_root}/data/Биохимические исследования/7/all",
        'bkg_count': 40,
        'likelihood': 1.00
    },
    'Биохимические исследования 7, старше 15': {
        'name': 'Биохимические исследования',
        'path': f"{dir_root}/data/Биохимические исследования/7/over15",
        'bkg_count': 12,
        'likelihood': 1.00
    },
    'Биохимические исследования 7, младше 15': {
        'name': 'Биохимические исследования',
        'path': f"{dir_root}/data/Биохимические исследования/7/under15",
        'bkg_count': 11,
        'likelihood': 1.00
    },
    'Биохимические исследования 9, все': {
        'name': 'Биохимические исследования',
        'path': f"{dir_root}/data/Биохимические исследования/9/all",
        'bkg_count': 30,
        'likelihood': 1.00
    },
    'Биохимические исследования 9, старше 15': {
        'name': 'Биохимические исследования',
        'path': f"{dir_root}/data/Биохимические исследования/9/over15",
        'bkg_count': 25,
        'likelihood': 1.00
    },
    'Биохимические исследования 9, младше 15': {
        'name': 'Биохимические исследования',
        'path': f"{dir_root}/data/Биохимические исследования/9/under15",
        'bkg_count': 12,
        'likelihood': 1.00
    },
    'Биохимические исследования 12': {
        'name': 'Биохимические исследования',
        'path': f"{dir_root}/data/Биохимические исследования/12",
        'bkg_count': 22,
        'likelihood': 1.00
    },
    'Биохимические исследования 23': {
        'name': 'Биохимические исследования',
        'path': f"{dir_root}/data/Биохимические исследования/23",
        'bkg_count': 22,
        'likelihood': 1.00
    },
    'Биохимические исследования 25, женщины': {
        'name': 'Биохимические исследования',
        'path': f"{dir_root}/data/Биохимические исследования/25/females",
        'bkg_count': 15,
        'likelihood': 1.00
    },
    'Биохимические исследования 25, мужчины': {
        'name': 'Биохимические исследования',
        'path': f"{dir_root}/data/Биохимические исследования/25/males",
        'bkg_count': 12,
        'likelihood': 1.00
    },
    
    'Половые гормоны 6, женщины': {
        'name': 'Половые гормоны',
        'path': f"{dir_root}/data/Половые гормоны/females/6",
        'bkg_count': 15,
        'likelihood': 0.70
    },
    'Половые гормоны 8, женщины': {
        'name': 'Половые гормоны',
        'path': f"{dir_root}/data/Половые гормоны/females/8",
        'bkg_count': 15,
        'likelihood': 0.70
    },
    'Половые гормоны 6, мужчины': {
        'name': 'Половые гормоны',
        'path': f"{dir_root}/data/Половые гормоны/males/6",
        'bkg_count': 15,
        'likelihood': 0.70
    },
    'Половые гормоны 10, мужчины': {
        'name': 'Половые гормоны',
        'path': f"{dir_root}/data/Половые гормоны/males/10",
        'bkg_count': 12,
        'likelihood': 0.70
    },
    
    'Гормоны 3, все': {
        'name': 'Гормоны',
        'path': f"{dir_root}/data/Гормоны/3/all",
        'bkg_count': 10,
        'likelihood': 0.70
    },
    'Гормоны 3, старше 18': {
        'name': 'Гормоны',
        'path': f"{dir_root}/data/Гормоны/3/over18",
        'bkg_count': 10,
        'likelihood': 0.70
    },
    'Гормоны 3, младше 18': {
        'name': 'Гормоны',
        'path': f"{dir_root}/data/Гормоны/3/under18",
        'bkg_count': 14,
        'likelihood': 0.70
    },
    'Гормоны 5': {
        'name': 'Гормоны',
        'path': f"{dir_root}/data/Гормоны/5",
        'bkg_count': 10,
        'likelihood': 0.70
    },
    'Гормоны 6': {
        'name': 'Гормоны',
        'path': f"{dir_root}/data/Гормоны/6",
        'bkg_count': 10,
        'likelihood': 0.70
    },
}

feats_all = ['Пол']
feats_pred_all = []
feats_input_all = []
feats_aux = []
for comp in components:
    components[comp]['data'] = pd.read_excel(f"{components[comp]['path']}/data.xlsx", index_col=0)
    components[comp]['feats'] = pd.read_excel(f"{components[comp]['path']}/feats.xlsx", index_col=0)
    components[comp]['results'] = pd.read_excel(f"{components[comp]['path']}/model/df.xlsx", index_col=0)
    components[comp]['metrics'] = pd.read_excel(f"{components[comp]['path']}/model/metrics.xlsx", index_col=0)
    components[comp]['model'] = TabularModel.load_model(f"{components[comp]['path']}/model")
    components[comp]['corrector'] = LinearBiasCorrector()
    comp_results = components[comp]['results']
    components[comp]['corrector'].fit(comp_results.loc[comp_results['Group'] == 'Train', feat_trgt].values, comp_results.loc[comp_results['Group'] == 'Train', 'Prediction'].values)
    res_cols = ['Group', 'Prediction', 'Error', 'Prediction Unbiased', 'Error Unbiased']
    components[comp]['data'].loc[components[comp]['data'].index, res_cols] = comp_results.loc[components[comp]['data'].index, res_cols]
    components[comp]['data_shap'] = components[comp]['data'].copy()
    
    feats = components[comp]['feats'].index.values
    feats = feats[feats != feat_trgt]
    feats_all += list(feats)
    feats_pred_all += [f"Предсказание {components[comp]['name']}", f"Возрастная Акселерация {components[comp]['name']}", f"Модель {components[comp]['name']}",]
    feats_all += [f"Модель {comp}", f"Предсказание {comp}", f"Возрастная Акселерация {comp}"]
    feats_aux += [f"Модель {comp}", f"Предсказание {comp}", f"Возрастная Акселерация {comp}"]
    feats_input_all += list(feats)
    
    components[comp]['feats_corr'] = pd.DataFrame(index=feats, columns=['Correlation'])
    for f in feats:
        components[comp]['feats_corr'].at[f, 'Correlation'], _ = scipy.stats.pearsonr(components[comp]['data'].loc[:, f].values, components[comp]['data'].loc[:, feat_trgt].values)

feats_all = list(dict.fromkeys(feats_all)) + list(dict.fromkeys(feats_pred_all))
feats_input_all = list(dict.fromkeys(feats_input_all))
feats_aux = list(dict.fromkeys(feats_aux))

for comp in components:
    print(f"{comp}: {components[comp]['data'].shape[0]}")

# Check local explainability

In [ ]:
path = 'E:/YandexDisk/Work/bbd/millennium/Результаты чекап/check/'

sample_folder = 'Добрица О. П' # 'Садчикова_raw' # '9077404592_Михайлова_Ольга_Игоревна_2024_raw'

samples = pd.read_pickle(f"{path}/{sample_folder}/data.pkl")
samples_imputed = pd.read_pickle(f"{path}/{sample_folder}/is_imputed.pkl")

trgt_id = samples.index[0]
local_exlp = {}

n_pos = samples.at[trgt_id, "Число моделей c положительной аккселерацией"]
n_neg = samples.at[trgt_id, "Число моделей c отрицательной аккселерацией"]
trgt_age = samples.at[trgt_id, feat_trgt]

components_exist = []
for comp in components:
    if samples.at[trgt_id, f"Модель {comp}"] == True:
        components_exist.append(comp)
        
for comp in components_exist:

    feats = components[comp]['feats'].index.values
    feats_corr = components[comp]['feats_corr']
    metrics = components[comp]['metrics']
    data_shap = components[comp]['data_shap']

    def predict_func(X):
        X_df = pd.DataFrame(data=X, columns=feats)
        y = components[comp]['model'].predict(X_df)[f'{feat_trgt}_prediction'].values
        y = components[comp]['corrector'].predict(y)
        return y

    mae = metrics.at['Test', 'mean_absolute_error_unbiased']
    # rho = metrics.at['Test', 'pearson_corrcoef_unbiased'] * components[comp]['likelihood']
    # rho = metrics.at['Test', 'pearson_corrcoef_unbiased'] * feats_corr['Correlation'].abs().max()
    rho = metrics.at['Test', 'pearson_corrcoef_unbiased'] * feats_corr['Correlation'].abs().max() * components[comp]['likelihood']

    bkg_count = components[comp]['bkg_count']

    trgt_pred_raw = samples.at[trgt_id, f"Предсказание {comp}"]

    data_closest_step_1 = data_shap.loc[data_shap['Error Unbiased'].abs() < mae * rho, :]
    data_closest_step_2 = data_closest_step_1.iloc[(data_closest_step_1['Prediction Unbiased'] - trgt_age).abs().argsort()[:bkg_count]]

    pathlib.Path(f"{path}/{sample_folder}/{comp}").mkdir(parents=True, exist_ok=True)
    
    rho = components[comp]['metrics'].at['Test', 'pearson_corrcoef_unbiased'] * components[comp]['feats_corr']['Correlation'].abs().max() * components[comp]['likelihood']
    components[comp]['data']['Биологический возраст'] = components[comp]['data'][feat_trgt] + rho * components[comp]['data']['Error Unbiased']
    components[comp]['data']['Аккселерация'] = rho * components[comp]['data']['Error Unbiased']
    fig, axs = plt.subplot_mosaic(
        [
            ['11', '12'],
        ],
        figsize=(6, 4),
        width_ratios=[4, 1.5],
        gridspec_kw={
            # "bottom": 0.14,
            # "top": 0.95,
            "wspace": 0.5,
            "hspace": 0.5,
        },
    )
    xy_min = components[comp]['data'][[feat_trgt, 'Биологический возраст']].min().min()
    xy_max = components[comp]['data'][[feat_trgt, 'Биологический возраст']].max().max()
    xy_ptp = xy_max - xy_min
    bisect = sns.lineplot(
        x=[xy_min - 0.1 * xy_ptp, xy_max + 0.1 * xy_ptp],
        y=[xy_min - 0.1 * xy_ptp, xy_max + 0.1 * xy_ptp],
        linestyle='--',
        color='black',
        linewidth=1.0,
        ax=axs['11']
    )
    regplot = sns.regplot(
        data=components[comp]['data'],
        x=feat_trgt,
        y='Биологический возраст',
        color='lightgray',
        scatter_kws=dict(
            linewidth=0.2,
            alpha=0.75,
            edgecolor="k",
            s=20,
        ),
        truncate=False,
        ax=axs['11']
    )
    scatter = sns.scatterplot(
        x=[trgt_age],
        y=[samples.at[trgt_id, feat_trgt] + samples.at[trgt_id, f"Возрастная Акселерация {comp}"]],
        color='crimson',
        linewidth=0.5,
        alpha=0.75,
        edgecolor="k",
        s=35,
        ax=axs['11'],
    )
    axs['11'].set_xlabel(f"Age")
    axs['11'].set_xlim(xy_min - 0.1 * xy_ptp, xy_max + 0.1 * xy_ptp)
    axs['11'].set_ylim(xy_min - 0.1 * xy_ptp, xy_max + 0.1 * xy_ptp)
    
    violin = sns.violinplot(
        data=components[comp]['data'],
        x=[0] * components[comp]['data'].shape[0],
        y='Аккселерация',
        color='lightgray',
        density_norm='width',
        saturation=0.75,
        linewidth=1.0,
        ax=axs['12'],
        legend=False,
    )
    swarm = sns.swarmplot(
        x=[0],
        y=[samples.at[trgt_id, f"Возрастная Акселерация {comp}"]],
        color='crimson',
        linewidth=0.5,
        ax=axs['12'],
        size=7,
        legend=False,
    )
    print(f"Возрастная Акселерация {comp}: {samples.at[trgt_id, f"Возрастная Акселерация {comp}"]}")
    axs['12'].set_ylabel('Возрастная акселерация')
    axs['12'].set_xlabel('')
    axs['12'].set(xticklabels=[]) 
    axs['12'].set(xticks=[]) 
    
    fig.savefig(f"{path}/{sample_folder}/{comp}/1.png", bbox_inches='tight', dpi=200)
    fig.savefig(f"{path}/{sample_folder}/{comp}/1.pdf", bbox_inches='tight')
    plt.close(fig)

    for f in feats:
        
        fig, axs = plt.subplot_mosaic(
            [
                ['11', '12'],
                ['21', '22'],
            ],
            figsize=(6, 10),
            height_ratios=[4, 4],
            width_ratios=[4, 1],
            gridspec_kw={
                # "bottom": 0.14,
                # "top": 0.95,
                "wspace": 0.05,
                "hspace": 0.25,
            },
        )
        
        x_min = components[comp]['data'][feat_trgt].min()
        x_max = components[comp]['data'][feat_trgt].max()
        x_ptp = x_max - x_min
        
        y_min = components[comp]['data'][f].min()
        y_max = components[comp]['data'][f].max()
        y_ptp = y_max - y_min
        
        regplot = sns.regplot(
            data=components[comp]['data'],
            x=feat_trgt,
            y=f,
            color='dodgerblue',
            scatter_kws=dict(
                linewidth=0.2,
                alpha=0.75,
                edgecolor="k",
                s=20,
            ),
            truncate=False,
            ax=axs['11']
        )
        scatter = sns.scatterplot(
            x=[trgt_age],
            y=[samples.at[trgt_id, f]],
            color='crimson',
            linewidth=0.5,
            alpha=0.75,
            edgecolor="k",
            s=35,
            ax=axs['11'],
        )
        axs['11'].set_xlabel(f"Age")
        axs['11'].set_xlim(x_min - 0.1 * x_ptp, x_max + 0.1 * x_ptp)
        axs['11'].set_ylim(y_min - 0.1 * y_ptp, y_max + 0.1 * y_ptp)
        axs['11'].set_title(fr"$\rho$ = {feats_corr.at[f, 'Correlation']:0.2f}")
        
        trgt_f_prctl = stats.percentileofscore(components[comp]['data'][f], samples.at[trgt_id, f])
        kdeplot = sns.kdeplot(
            data=components[comp]['data'],
            y=f,
            color='gray',
            linewidth=1,
            cut=0,
            ax=axs['12']
        )
        axs['12'].set_ylim(y_min - 0.1 * y_ptp, y_max + 0.1 * y_ptp)
        axs['12'].set_yticklabels([])
        axs['12'].set_ylabel('')
        kdeline = axs['12'].lines[0]
        xs = kdeline.get_xdata()
        ys = kdeline.get_ydata()
        axs['12'].fill_betweenx(ys, 0, xs, where=(ys <= samples.at[trgt_id, f]), interpolate=True, facecolor='mediumpurple', alpha=0.7)
        axs['12'].fill_betweenx(ys, 0, xs, where=(ys >= samples.at[trgt_id, f]), interpolate=True, facecolor='gold', alpha=0.7)
        axs['12'].hlines(samples.at[trgt_id, f], 0, max(xs) * 1.2, color='gray', linewidth=2)
        axs['12'].text(0.9 * max(xs), np.mean([min(ys), samples.at[trgt_id, f]]), f"{trgt_f_prctl:0.1f}%",
            fontstyle="oblique", color="black", ha="center", va="center")
        axs['12'].text(0.9 * max(xs), np.mean([max(ys), samples.at[trgt_id, f]]), f"{100 - trgt_f_prctl:0.1f}%",
            fontstyle="oblique", color="black", ha="center", va="center")
        
        regplot = sns.regplot(
            data=data_closest_step_2,
            fit_reg=False,
            x=feat_trgt,
            y=f,
            color='lawngreen',
            scatter_kws=dict(
                linewidth=0.2,
                alpha=0.75,
                edgecolor="k",
                s=20,
            ),
            ax=axs['21']
        )
        scatter = sns.scatterplot(
            x=[trgt_age],
            y=[samples.at[trgt_id, f]],
            color='crimson',
            linewidth=0.5,
            alpha=0.75,
            edgecolor="k",
            s=35,
            ax=axs['21'],
        )
        axs['21'].set_xlabel(f"Age")
        axs['21'].set_xlim(x_min - 0.1 * x_ptp, x_max + 0.1 * x_ptp)
        axs['21'].set_ylim(y_min - 0.1 * y_ptp, y_max + 0.1 * y_ptp)
        axs['21'].set_title(f"Closest samples")
        
        trgt_f_prctl = stats.percentileofscore(data_closest_step_2[f], samples.at[trgt_id, f])
        kdeplot = sns.kdeplot(
            data=data_closest_step_2,
            y=f,
            color='gray',
            linewidth=1,
            cut=0,
            ax=axs['22']
        )
        axs['22'].set_ylim(y_min - 0.1 * y_ptp, y_max + 0.1 * y_ptp)
        axs['22'].set_yticklabels([])
        axs['22'].set_ylabel('')
        kdeline = axs['22'].lines[0]
        xs = kdeline.get_xdata()
        ys = kdeline.get_ydata()
        axs['22'].fill_betweenx(ys, 0, xs, where=(ys <= samples.at[trgt_id, f]), interpolate=True, facecolor='mediumpurple', alpha=0.7)
        axs['22'].fill_betweenx(ys, 0, xs, where=(ys >= samples.at[trgt_id, f]), interpolate=True, facecolor='gold', alpha=0.7)
        axs['22'].hlines(samples.at[trgt_id, f], 0, max(xs) * 1.2, color='gray', linewidth=2)
        axs['22'].text(0.9 * max(xs), np.mean([min(ys), samples.at[trgt_id, f]]), f"{trgt_f_prctl:0.1f}%",
            fontstyle="oblique", color="black", ha="center", va="center")
        axs['22'].text(0.9 * max(xs), np.mean([max(ys), samples.at[trgt_id, f]]), f"{100 - trgt_f_prctl:0.1f}%",
            fontstyle="oblique", color="black", ha="center", va="center")
        
        fig.savefig(f"{path}/{sample_folder}/{comp}/{slugify(f)}.png", bbox_inches='tight', dpi=200)
        fig.savefig(f"{path}/{sample_folder}/{comp}/{slugify(f)}.pdf", bbox_inches='tight')
        plt.close(fig)
        
        
    explainer = shap.SamplingExplainer(predict_func, data_closest_step_2.loc[:, feats].values)
    shap_values = explainer.shap_values(samples.loc[[trgt_id], feats].values)[0]
    shap_values = shap_values * (trgt_pred_raw - trgt_age) / (trgt_pred_raw - explainer.expected_value)
    print(f"SHAP values difference: {sum(shap_values) - (trgt_pred_raw - trgt_age)}")
    # shap_values = shap_values * rho / max(n_pos - 1, n_neg - 1, 1)
    shap_values = shap_values * rho

    # SHAP values correction 1
    shap_corr_thld = 3.0
    shap_corr_to = 1.0
    shap_mean_abs = np.mean(np.abs(shap_values))
    if shap_mean_abs > shap_corr_thld and abs(sum(shap_values)) < shap_corr_thld:
        print('SHAP values correction 1')
        shap_pos_ids = np.argwhere(shap_values >= 0).ravel()
        shap_neg_ids = np.argwhere(shap_values < 0).ravel()
        
        shap_pos_sum_abs = np.sum(np.abs(shap_values[shap_pos_ids]))
        shap_neg_sum_abs = np.sum(np.abs(shap_values[shap_neg_ids]))
        
        shap_sum_abs_from = np.sum(np.abs(shap_values))
        shap_sum_abs_to = shap_corr_to * len(shap_values)
        
        shap_corr_diff = shap_sum_abs_from - shap_sum_abs_to
        
        for pos_id in shap_pos_ids:
            curr_part = abs(shap_values[pos_id]) / shap_pos_sum_abs
            shap_values[pos_id] -= curr_part * shap_corr_diff * 0.5
        for neg_id in shap_neg_ids:
            curr_part = abs(shap_values[neg_id]) / shap_neg_sum_abs
            shap_values[neg_id] += curr_part * shap_corr_diff * 0.5
            
    # SHAP values correction 2
    shap_corr_max_thld = 1.0
    shap_corr_max_to = 0.95
    shap_max_abs = np.max(np.abs(shap_values))
    shap_abs_sum = np.abs(np.sum(shap_values))
    if shap_max_abs > shap_corr_max_thld * shap_abs_sum:
        print('SHAP values correction 2')
        shap_pos_ids = np.argwhere(shap_values >= 0).ravel()
        shap_neg_ids = np.argwhere(shap_values < 0).ravel()
        
        shap_pos_sum_abs = np.sum(np.abs(shap_values[shap_pos_ids]))
        shap_neg_sum_abs = np.sum(np.abs(shap_values[shap_neg_ids]))
        
        shap_corr_diff = (shap_max_abs - shap_corr_max_thld) * shap_corr_max_to
        
        for pos_id in shap_pos_ids:
            curr_part = abs(shap_values[pos_id]) / shap_pos_sum_abs
            shap_values[pos_id] -= curr_part * shap_corr_diff
        for neg_id in shap_neg_ids:
            curr_part = abs(shap_values[neg_id]) / shap_neg_sum_abs
            shap_values[neg_id] += curr_part * shap_corr_diff

    df_comp = pd.DataFrame(index=feats, columns=['SHAP', 'Values', 'Correlation', 'Percentile', 'Class', 'Consistent', 'Show'])
    df_comp['SHAP'] = shap_values
    df_comp.sort_values(by='SHAP', key=abs, inplace=True)
    df_comp.loc[df_comp.index.values, 'Values'] = samples.loc[trgt_id, df_comp.index.values].values
    df_comp.loc[df_comp.index.values, 'Correlation'] = feats_corr.loc[df_comp.index.values, 'Correlation']
    for f in df_comp.index.values:
        df_comp.at[f, 'Percentile'] = scipy.stats.percentileofscore(data_closest_step_2.loc[:, f].values, samples.at[trgt_id, f])
        df_comp.at[f, 'Class'] = int(df_comp.at[f, 'Percentile'] // 33.333334)
        if (
            ((df_comp.at[f, 'Correlation'] > 0) & (df_comp.at[f, 'SHAP'] > 0) & (df_comp.at[f, 'Class'] == 2)) or \
            ((df_comp.at[f, 'Correlation'] > 0) & (df_comp.at[f, 'SHAP'] < 0) & (df_comp.at[f, 'Class'] == 0)) or \
            ((df_comp.at[f, 'Correlation'] < 0) & (df_comp.at[f, 'SHAP'] > 0) & (df_comp.at[f, 'Class'] == 0)) or \
            ((df_comp.at[f, 'Correlation'] < 0) & (df_comp.at[f, 'SHAP'] < 0) & (df_comp.at[f, 'Class'] == 2))
            ):
            df_comp.at[f, 'Consistent'] = True
        else:
            df_comp.at[f, 'Consistent'] = False
        if (
            (df_comp.at[f, 'Consistent'] == True) or \
            ((df_comp.at[f, 'Consistent'] == False) & (df_comp.at[f, 'Class'] == 1))
            ):
            df_comp.at[f, 'Show'] = True
        else:
            df_comp.at[f, 'Show'] = False
        # If SHAP values too small
        if abs(df_comp.at[f, 'SHAP']) < 0.1:
            df_comp.at[f, 'Show'] = False
        # Check if feature value was imputed
        if samples_imputed.at[trgt_id, f]:
            df_comp.at[f, 'Show'] = False
    df_comp['Class'] = df_comp['Class'].replace({0: 'Понижен', 1: 'Средний', 2: 'Повышен'})

    local_exlp[comp] = {
        'df': df_comp,
        'age_acceleration': samples.at[trgt_id, f"Возрастная Акселерация {comp}"],
    }

    df_comps = pd.DataFrame(index=list(local_exlp.keys()), columns=['age_acceleration'])    
    for comp in local_exlp:
        df_comps.at[comp, 'age_acceleration'] = local_exlp[comp]['age_acceleration']
    df_comps['abs_age_acceleration'] = df_comps['age_acceleration'].abs()
    df_comps.sort_values(["abs_age_acceleration"], ascending=[True], inplace=True)
    df_comps['cumsum'] = df_comps['age_acceleration'].cumsum()
    comps_sorted = df_comps.index.values

    n_show = 50
    color_regular = 'dodgerblue'
    color_pos_first = 'crimson'
    color_pos_second = 'orangered'
    color_neg_first = 'forestgreen'
    color_neg_second = 'chartreuse'
    color_neutral = 'dimgray'

    df_comp = local_exlp[comp]['df']
    aa_comp = local_exlp[comp]['age_acceleration']

    df_comp.sort_values(['SHAP'], key=abs, inplace=True)
    df_comp['SHAP cumsum'] = df_comp['SHAP'].cumsum()

    fig = make_subplots(rows=1, cols=2, shared_yaxes=True, shared_xaxes=False, column_widths=[2.5, 0.5], horizontal_spacing=0.2, subplot_titles=['', 'Корреляция с возрастом'])

    if df_comp.shape[0] <= n_show:

        fig.add_trace(
            go.Waterfall(
                hovertext=["Итоговая акселерация"],
                orientation="h",
                measure=['relative'],
                y=[df_comp.shape[0] + 1],
                x=[aa_comp],
                base=0,
                text=[f"+{aa_comp:0.2f}" if aa_comp > 0 else f"{aa_comp:0.2f}"],
                textposition = "auto",
                decreasing = {"marker":{"color": color_neg_first, "line": {"color": "black", "width": 1}}},
                increasing = {"marker":{"color": color_pos_first, "line": {"color": "black", "width": 1}}},
                totals= {"marker":{"color": color_neutral, "line": {"color": "black", "width": 1}}},
                connector={
                    "mode": "between",
                    "line": {"width": 1, "color": "black", "dash": "dot"},
                },
            ),
            row=1,
            col=1,
        )
        fig.add_trace(
            go.Waterfall(
                hovertext=df_comp.index.values,
                orientation="h",
                measure=["relative"] * len(df_comp.index.values),
                y=list(range(df_comp.shape[0])),
                x=df_comp['SHAP'].values,
                base=0,
                text=[f"+{x:0.2f}" if x > 0 else f"{x:0.2f}" for x in df_comp['SHAP'].values],
                textposition = "auto",
                decreasing = {"marker":{"color": color_neg_second, "line": {"color": "black", "width": 1}}},
                increasing = {"marker":{"color": color_pos_second, "line": {"color": "black", "width": 1}}},
                connector={
                    "mode": "between",
                    "line": {"width": 1, "color": "black", "dash": "solid"},
                },
            ),
            row=1,
            col=1,
        )
        fig.update_traces(row=1, col=1, showlegend=False)
        fig.update_yaxes(
            row=1,
            col=1,
            automargin=True,
            tickmode="array",
            tickvals=list(range(df_comp.shape[0])) + [df_comp.shape[0] + 1],
            ticktext=[f"{x} = {df_comp.at[x, 'Values']:0.2f} ({df_comp.at[x, 'Class']})" for x in df_comp.index] + ['Итоговая акселерация'],
            tickfont=dict(size=18),
        )
        fig.update_xaxes(
            row=1,
            col=1,
            automargin=True,
            title='Возрастная акселерация',
            titlefont=dict(size=20),
        )

        fig.add_trace(
            go.Heatmap(
                x=['Возраст'],
                y=list(range(df_comp.shape[0])),
                z=[[x] for x in df_comp['Correlation'].values],
                text=[[f"{x:0.2f}"] for x in df_comp['Correlation'].values],
                texttemplate="%{text}",
                zmin=-1,
                zmax=1,
                colorbar_y=df_comp.shape[0] / (df_comp.shape[0] + 2) * 0.5,
                colorscale='Temps'
            ),
            row=1,
            col=2,
        )

        fig.update_layout(
            title=f"{components[comp]['name']}",
            titlefont=dict(size=25),
            template="none",
            width=1500,
            height=df_comp.shape[0]*40 + 100,
            margin=go.layout.Margin(l=20, r=20, b=20, t=50),
        )
    else:
        
        n_feats_show = df_comp.loc[df_comp['Show'] == True, :].shape[0]
        if n_feats_show > n_show:
            feats_show = df_comp.index[df_comp['Show'] == True].tolist()
            feats_show = feats_show[-n_show:]
        else:
            feats_show = df_comp.index.tolist()
            feats_show = feats_show[-n_show:]
        
        df_show = df_comp.loc[feats_show, :]
        df_not_show = df_comp.loc[list(set(df_comp.index.values) - set(feats_show)), :]
        df_show.sort_values(['SHAP'], key=abs, inplace=True)
        df_not_show.sort_values(['SHAP'], key=abs, inplace=True)
        shap_not_show_sum = df_not_show['SHAP'].sum()

        fig.add_trace(
            go.Waterfall(
                hovertext=["Итоговая акселерация"],
                orientation="h",
                measure=['relative'],
                y=[df_show.shape[0] + 1],
                x=[aa_comp],
                base=0,
                text=[f"+{aa_comp:0.2f}" if aa_comp > 0 else f"{aa_comp:0.2f}"],
                textposition = "auto",
                decreasing = {"marker":{"color": color_neg_first, "line": {"color": "black", "width": 1}}},
                increasing = {"marker":{"color": color_pos_first, "line": {"color": "black", "width": 1}}},
                totals = {"marker":{"color": color_neutral, "line": {"color": "black", "width": 1}}},
                connector={
                    "mode": "between",
                    "line": {"width": 1, "color": "black", "dash": "dot"},
                },
            ),
            row=1,
            col=1,
        )
        fig.add_trace(
            go.Waterfall(
                hovertext=df_show.index.values,
                orientation="h",
                measure=["relative"] * len(df_show.index.values),
                y=[-1] + list(range(df_show.shape[0])),
                x=[shap_not_show_sum] + df_show['SHAP'].tolist(),
                base=0,
                text=[f"+{shap_not_show_sum:0.2f}" if shap_not_show_sum > 0 else f"{shap_not_show_sum:0.2f}"] + [f"+{x:0.2f}" if x > 0 else f"{x:0.2f}" for x in df_show['SHAP'].values],
                textposition = "auto",
                decreasing = {"marker":{"color": color_neg_second, "line": {"color": "black", "width": 1}}},
                increasing = {"marker":{"color": color_pos_second, "line": {"color": "black", "width": 1}}},
                connector={
                    "mode": "between",
                    "line": {"width": 1, "color": "black", "dash": "solid"},
                },
            ),
            row=1,
            col=1,
        )
        fig.update_traces(row=1, col=1, showlegend=False)
        fig.update_yaxes(
            row=1,
            col=1,
            automargin=True,
            tickmode="array",
            tickvals=[-1] + list(range(df_show.shape[0])) + [df_show.shape[0] + 1],
            ticktext=['Остальные признаки'] + [f"{x} = {df_show.at[x, 'Values']:0.2f} ({df_show.at[x, 'Class']})" for x in df_show.index] + ['Итоговая акселерация'],
            tickfont=dict(size=18),
        )
        fig.update_xaxes(
            row=1,
            col=1,
            automargin=True,
            title='Возрастная акселерация',
            titlefont=dict(size=20),
        )

        fig.add_trace(
            go.Heatmap(
                x=['Возраст'],
                y=list(range(df_show.shape[0])),
                z=[[x] for x in df_show['Correlation'].values],
                text=[[f"{x:0.2f}"] for x in df_show['Correlation'].values],
                texttemplate="%{text}",
                zmin=-1,
                zmax=1,
                colorbar_y=(df_show.shape[0] + 1) / (df_show.shape[0] + 3) * 0.5,
                colorscale='Temps'
            ),
            row=1,
            col=2,
        )

        fig.update_layout(
            title=f"{components[comp]['name']}",
            titlefont=dict(size=25),
            template="none",
            width=1500,
            height=(df_show.shape[0] + 1) * 40 + 100,
            margin=go.layout.Margin(l=20, r=20, b=20, t=50),
        )

    fig.update_xaxes(
        row=1,
        col=2,
        tickvals=[],
        automargin=True,
        showgrid=False,
        showline=False,
        zeroline=False,
        showticklabels=False,
    )
    fig.update_yaxes(
        row=1,
        col=2,
        tickvals=[],
        automargin=True,
        showgrid=False,
        showline=False,
        zeroline=False,
        showticklabels=False,
    )

    fig.write_image(f"{path}/{sample_folder}/{comp}/0.png", scale=2)